<a href="https://colab.research.google.com/github/AvantiShri/oceanography_colab_notebooks/blob/master/classes/estimating_npp_from_satellite_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install necessary python packages
!pip install netCDF4
!pip install wget

    100% |████████████████████████████████| 4.0MB 6.3MB/s 
    100% |████████████████████████████████| 307kB 27.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
import netCDF4
import os
from netCDF4 import Dataset
import numpy as np
import h5py

def radians(deg):
  return (deg/180.0)*(np.pi)

def degrees(rad):
  return 180.0*(rad/(np.pi))

def area_of_patch(lat1,lat2,lon1,lon2):
  earth_radius = 6378100 #meters
  return (np.abs(np.sin(radians(lat2)) - np.sin(radians(lat1)))
          *2*np.pi*(earth_radius**2)*(np.abs(lon2-lon1)/360.0))

def daylength_calc(julian_day, lat):
  date_angle_rad = radians(360.0*(julian_day/365.0))
  
  decl_deg = (0.39637
              -22.9133*np.cos(date_angle_rad)
              +4.02543*np.sin(date_angle_rad)
              -0.3872*np.cos(2*date_angle_rad)
              +0.052*np.sin(2*date_angle_rad))
  decl_rad = -radians(decl_deg)
  lat_rad = -radians(lat)
  
  tan_prod = -np.tan(lat_rad)*np.tan(decl_rad)
  if (np.isnan(np.arccos(tan_prod))==False):
    return 0.133*degrees(np.arccos(tan_prod))
  else:
    if (np.abs(tan_prod)==tan_prod):
      return 0
    else:
      return 24

def format_day_string(day):
  return "".join(["0" for i in range(3-len(str(day)))]) + str(day)

def read_map_dataset(file, data_key):
  dataset = Dataset(file)

  data = np.array(dataset[data_key])
  mask = (data > dataset[data_key]._FillValue)
  data[mask==False] = np.nan
  latitudes = np.array(dataset['lat'])
  longitudes = np.array(dataset['lon'])
  assert len(latitudes) == data.shape[0]
  assert len(longitudes) == data.shape[1]
  return data, mask, latitudes, longitudes
  
def npp_integral(Pmax, K, z, Eo, Ec):
  
  #Half Saturation Constant; Ek = 30 µEin m-2 s-1
  Ek = 30 #µEin m-2 s-1
  
  #5.	Then calculate NPP (Pn) using the following equation that accounts
  # for phytoplankton photophysiology and changes in the light
  # environment with depth (Ez):
  # Pn = [Pmax * (Ez - Ec)] / [Ek + (Ez - Ec)]
  
  ##We will take the integral.
  ##Rearranging:
  # Pn = [Pmax]/[Ek/(Ez - Ec) + 1]
  ##Substituting Beer's law: Ez = E0 * exp(-Kz)
  # Pn = [Pmax]/[Ek/(E0*exp(-Kz) - Ec) + 1]
  ##After plugging into Wolfram Alpha:
  # (Pmax)/(K(Ec-Ek)) * [Ek*log(Ek - Ec + E0*exp(-Kz)) + Ec*K*z]
  
  ##Confirm the integral is right by taking the derivative:
  # d/dz (Pmax/(K(Ec-Ek)))*[Ek*log(Ek - Ec + E0*exp(-Kz)) + Ec*K*z]
  ##Apply the chain rule:
  # = (Pmax/(K(Ec-Ek)))*[-Ek*E0*K*exp(-Kz)/[Ek - Ec + E0*exp(-Kz)] + Ec*K]
  ##Cancel out K
  # = (Pmax/(Ec-Ek))*[-Ek*E0*exp(-Kz)/[Ek - Ec + E0*exp(-Kz)] + Ec]
  ##Replace E0*exp(-Kz) with Ez 
  # = (Pmax/(Ec-Ek))*[-Ek*Ez/[Ek - Ec + Ez] + Ec]
  ##Unify the denominator into a single term
  # = (Pmax/(Ec-Ek))*[(-Ek*Ez + Ec*(Ek - Ec + Ez)) /[Ek - Ec + Ez]]
  ##Refactor the denominator
  # = (Pmax/(Ec-Ek))*[(-Ek*Ez + Ec*Ez + Ec*(Ek - Ec)) /[Ek - Ec + Ez]]
  # = (Pmax/(Ec-Ek))*[((Ec-Ek)*Ez + Ec*(Ek - Ec)) /[Ek - Ec + Ez]]
  # = (Pmax/(Ec-Ek))*[((Ec-Ek)*Ez - Ec*(Ec - Ek)) /[Ek - Ec + Ez]]
  ##Cancel out Ec-Ek
  # = Pmax*[(Ez - Ec) /[Ek - Ec + Ez]]
  ## It's the same!
  # = Pmax*(Ez - Ec)/[Ek - (Ez-Ec)]

  return ((Pmax/(K*(Ec-Ek)))
          *((Ek*np.log(Ek - Ec + Eo*np.exp(-K*z))) + Ec*K*z))


def process_data_for_month(start_year, start_day, end_year, end_day,
                           save_intermediate_data=False):
  
  print("On",start_year,start_day,"to",end_year,end_day)
  
  date_string = (str(start_year)+format_day_string(start_day)
                 +str(end_year)+format_day_string(end_day))
  
  chla_file = "A"+date_string+".L3m_MO_CHL_chlor_a_4km.nc"
  par_file = "A"+date_string+".L3m_MO_PAR_par_4km.nc"
  sst_file = "A"+date_string+".L3m_MO_SST_sst_4km.nc"
  
  #download the files
  for file in [chla_file, par_file, sst_file]:
    if (os.path.isfile(file)==False):
      print("Downloading",file)
      wget.download(url="https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/"+file,
                    out=file)
      
  chla_data, chla_mask, chla_lat, chla_lon = read_map_dataset(
                                             file=chla_file, data_key='chlor_a')
  par_data, par_mask, par_lat, par_lon = read_map_dataset(
                                             file=par_file, data_key='par')
  sst_data, sst_mask, sst_lat, sst_lon = read_map_dataset(
                                             file=sst_file, data_key='sst')
  #os.remove(chla_file)
  #os.remove(par_file)
  #os.remove(sst_file)
  
  lats = chla_lat
  lons = chla_lon
  #double-check that the latitudes and longitudes are consistent across all the
  # datasets
  assert np.sum(np.abs(lats-chla_lat))==0, np.sum(np.abs(lats-chla_lat))
  assert np.sum(np.abs(lats-par_lat))==0, np.sum(np.abs(lats-par_lat))
  assert np.sum(np.abs(lats-sst_lat))==0, np.sum(np.abs(lats-sst_lat))
  assert np.sum(np.abs(lons-chla_lon))==0, np.sum(np.abs(lons-chla_lon))
  assert np.sum(np.abs(lons-par_lon))==0, np.sum(np.abs(lons-par_lon))
  assert np.sum(np.abs(lons-sst_lon))==0, np.sum(np.abs(lons-sst_lon))

  
  #combined_mask is the mask that represents all unavailable values.
  # "True" means the value is available, "False" means it's not.
  combined_mask = chla_mask*par_mask*sst_mask
  
  
  #Diffuse Attenuation Coefficient; K = 0.04 + 0.05 * [Chl a]0.681 (m-1)
  K = 0.04 + 0.05*np.power(chla_data,0.681) #m-1
  
  
  #Assume the mixed layer depth equals the euphotic depth.
  # Think about what the euphotic depth is and how you will calculate it.
  #Ans: euphotic depth is generally taken to be the depth at which irradiance
  # is 1% of surface levels. Some exceptions are made in oligotrophic gyres,
  # but these gyres don't have a lot of primary production so we will just
  # use 1%. But we also need to make sure that we are above the compensation
  # intensity
  #Compensation Intensity; Ec = 10 µEin m-2 s-1
  Ec = 10 #µEin m-2 s-1
  euphotic_depth_light_frac = 0.01
  
  #1a-c:	Estimate the mean surface irradiance, Eo (µEin m-2 s-1),
  # for each of your stations and seasons.
  #Answer: this is par_data; we obtained it directly
  # but the par data is for the whole day, so we are going to use some
  # formulas from the activity to estimate the noon irradiance
  
  Eo_daily_integral = par_data*1000000 #convert from Einstiens to µEin 
  
  #Get the mean daylength from the period start_day to end_day
  lat_mean_daylengths = np.mean(np.array([[
    daylength_calc(julian_day=y, lat=x) for x in lats]
    for y in range(start_day, end_day+1)]), axis=0)
  
  #divide by (24 - photoperiod)*3600 to get average irradiance per second.
  Eo_avg_persec = Eo_daily_integral/(
                   1e-7 + (24 - lat_mean_daylengths[:,None])*3600)
  
  #assuming irradiance varies as described in handout
  # from noon to end of photoperiod, we can divide by 0.5 to
  # get irradiance at noon
  Eo = 2*Eo_avg_persec
  
  #2.	You will be calculating depth-integrated NPP based
  # on surface conditions of Chl a, light, and temperature.
  # Before you get started, discuss with your group
  # how many depths you should use.  
  #Answer: we need to solve for the depth that has 1% of surface irradiance
  # or has intensity equal to the compensation intensity, whichever is smaller.
  # Beer’s law:  Ez = E0 * exp(-Kz)
  # exp(-Kz) = 0.01
  # -Kz = log(0.01)
  # z = -log(0.01)/K
  euphotic_depth = np.minimum(-np.log(euphotic_depth_light_frac),
                              -np.log(Ec/Eo))/K #in meters
  
  #3.	Calculate light penetration through the water column using the
  # Beer-Lambert law: Beer’s law:  Ez = E0 * exp(-Kz)
  #Ans: we will just incorporate Beer's law later as part of the integral
  
  #4. Calculate maximum rate of photosynthesis, Pmax (mg C mg-1 Chl a h-1),
  # from sea surface temperature (T) using: Pmax = 1.8 * exp(0.0633*T) 
  Pmax = 1.8*np.exp(0.0633*sst_data)*chla_data #Assuming T is in deg C
  
  #5.	Then calculate NPP (Pn) using the following equation that accounts
  # for phytoplankton photophysiology and changes in the light
  # environment with depth (Ez):
  # Pn = [Pmax * (Ez - Ec)] / [Ek + (Ez - Ec)]
  
  npp_noon = (npp_integral(Pmax=Pmax, K=K, z=euphotic_depth, Eo=Eo, Ec=Ec)
              - npp_integral(Pmax=Pmax, K=K, z=0, Eo=Eo, Ec=Ec))
  
  #Step 3
  # Use the spreadsheet "daylength_calc.xls" to calculate the hours of
  # sunlight per day (daylength) at a specific latitude for a specific day.
  # After you’ve calculated this, double check your daylength results
  # to see if this makes sense with what you know about your region’s
  # latitude and hemisphere
  
  
  
  #1. Integrate NPP over the photoperiod (F) to obtain daily NPP,
  # taking into account the latitude and season. Since E0 is measured
  # at noon, scale your NPP value using the relationship:
  # PP (mg C m-2 d-1) = 0.5 F*(PPnoon – PPnoon+0.5F)
  
  # We assume PP_{noon+0.5F} = 0 due to no irradiance, so:
  npp_per_day = 0.5*lat_mean_daylengths[:,None]*npp_noon
  npp_whole_period = npp_per_day*(end_day - start_day)
  
  if (save_intermediate_data):
    file_name = ("data_"
                  +str(start_year)+"_"+str(start_day)
                  +"_to_"+str(end_year)+"_"+str(end_day))+".h5"
    if (os.path.isfile(file_name)):
      os.remove(file_name)
    f = h5py.File(file_name)
    f.create_dataset("chla", data=chla_data)
    f.create_dataset("Eo", data=Eo)
    f.create_dataset("sst", data=sst_data)
    f.create_dataset("lats", data=lats)
    f.create_dataset("lons", data=lons)
    f.create_dataset("mask", data=combined_mask),
    f.create_dataset("K", data=K)
    f.create_dataset("euphotic_depth", data=euphotic_depth)
    f.create_dataset("Pmax", data=Pmax)
    f.create_dataset("npp_noon", data=npp_noon)
    f.create_dataset("lat_mean_daylengths", data=lat_mean_daylengths)
    f.create_dataset("npp_whole_period", data=npp_whole_period)
    f.close()
  
  return npp_whole_period, lats, lons

  #print(np.nanmax(chla_data), np.nanmin(chla_data))
  #from matplotlib import pyplot as plt
  #import seaborn as sns
  #sns.heatmap(np.log(1+chla_data), xticklabels=False, yticklabels=False)
  #plt.imshow(chla_data[1500:2000, 0:1000], vmin=np.nanmin(chla_data), vmax=np.nanmax(chla_data))
  #plt.show()


In [ ]:
#date ranges for the months in terms of julian days
julian_date_ranges = [
    (1, 31),
    (32, 59),
    (60, 90),
    (91, 120),
    (121, 151),
    (152, 181),
    (182, 212),
    (213, 243),
    (244, 273),
    (274, 304),
    (305, 334),
    (335, 365)  
]

npps_list = []
lats_list = []
lons_list = []

for (start_day, end_day) in julian_date_ranges:
  npp_whole_period, lats, lons =\
    process_data_for_month(2003, start_day, 2003, end_day)
  npps_list.append(npp_whole_period)
  lats_list.append(lats)
  lons_list.append(lons)

On 2003 1 to 2003 31


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:188: RuntimeWarning: divide by zero encountered in true_divide


On 2003 32 to 2003 59
On 2003 60 to 2003 90
On 2003 91 to 2003 120
On 2003 121 to 2003 151
On 2003 152 to 2003 181
On 2003 182 to 2003 212
On 2003 213 to 2003 243
On 2003 244 to 2003 273
On 2003 274 to 2003 304
On 2003 305 to 2003 334
On 2003 335 to 2003 365


In [ ]:
#sum up the npp over all the months
sum_npp = np.sum(npps_list, axis=0)

In [ ]:
#verify that all the latitudes and longitudes are compatible
for lats in lats_list:
  assert np.sum(np.abs(lats-lats_list[0]))==0
for lons in lons_list:
  assert np.sum(np.abs(lons-lons_list[0]))==0

lats = lats_list[0]
lons = lons_list[0]

In [ ]:
#create an array that stores the area of a particular latitude-longitude
# patch, accounting for the curvature of the earth
area_grid = area_of_patch(lat1=lats[:,None],
                          lat2=np.array(list(lats[1:])+[-90.0])[:,None],
                          lon1=lons[None,:],
                          lon2=np.array(list(lons[1:])+[180.0])[None,:] )
print("Sanity check - total area of earth in m2:",np.sum(area_grid))

Sanity check - total area of earth in m2: 511172633667834.9


In [ ]:
#Create a mask that selects coordinates that correspond to the north pacific
north_pacific_mask = (((lats > 0)*(lats < 60))[:,None])*((np.abs(lons) > 105)[None,:])

In [ ]:
total_production = np.nansum(sum_npp*area_grid*north_pacific_mask)
print(total_production/(1.0e15)) #in Tg C per year

3970.2361367876288


In [ ]:
average_production_m2 = total_production/np.nansum(area_grid*north_pacific_mask)
print(average_production_m2/1000.0) #in gC/m2 per year

43.04232671515516


In [ ]:
np.nansum(area_grid*north_pacific_mask)

92240276950216.83

In [ ]:
world_production = np.nansum(sum_npp*area_grid)
print(world_production/(1.0e15)) #in Tg C per year

20795.250697877585


General overview of reading in NetCDF4 files...

In [ ]:

from netCDF4 import Dataset

dataset = Dataset("A20030012003031.L3m_MO_CHL_chlor_a_4km.nc")
print("types of entries:",list(dataset.variables.keys()))

types of entries: ['chlor_a', 'lat', 'lon', 'palette']


In [ ]:
import numpy as np

chla_data = np.array(dataset['chlor_a'])
lats = np.array(dataset['lat'])
lons = np.array(dataset['lon'])
print("chla data shape:",chla_data.shape)
print("lats shape:",lats.shape)
print("lons shape:",lons.shape)

chla data shape: (4320, 8640)
lats shape: (4320,)
lons shape: (8640,)


In [ ]:
print("latitude:", lats[1000], "longitude:", lons[500],
      "value:", chla_data[1000, 500])

latitude: 48.3125 longitude: -159.14584 value: 0.29979503


In [ ]:
print("masked value:", dataset['chlor_a']._FillValue)
print(chla_data[0, 500])

masked value: -32767.0
-32767.0
